# 🛡️ AI Fraud Risk Analyzer — Colab Edition

Welcome to your interactive fraud detection playground!  
This notebook blends rule-based scoring with LLM-powered explanations to help you analyze financial transactions in real time.

---

## 🚀 What’s Inside

- 🔍 **Fraud Scoring Engine** — Evaluates risk based on amount, merchant, method, and timing
- 🧠 **LLM Explanation** — Uses a large language model to explain why a transaction might be risky
- 🎲 **Simulation Module** — Generates synthetic transactions for testing
- 📊 **Insights Dashboard** — View fraud rates and transaction summaries

---

## 🧠 Powered By

- 🤖 Hugging Face Transformers  
- 📚 Mistral-7B-Instruct (or your preferred model)  
- 🐍 Python + Pandas  
- 🧪 Google Colab

---

> ⚠️ Note: If you're using a gated model like `mistralai/Mistral-7B-Instruct-v0.2`, make sure you've requested access and authenticated with your Hugging Face token.

Let’s dive in and make fraud detection explainable, teachable, and deployable — all from your browser.

In [ ]:
!pip install transformers --quiet

from datetime import datetime
import random
import pandas as pd
from transformers import pipeline

explainer = pipeline("text-generation", model="tiiuae/falcon-7b-instruct")

In [ ]:
transactions = [
    {"id": 1, "user_id": "alice", "amount": 150.0, "merchant": "amazon", "method": "credit_card"},
    {"id": 2, "user_id": "bob", "amount": 2500.0, "merchant": "suspicious_store", "method": "wire_transfer"},
    {"id": 3, "user_id": "charlie", "amount": 50.0, "merchant": "starbucks", "method": "debit_card"},
]

In [ ]:
def score_transaction(tx):
    amount = tx["amount"]
    merchant = tx["merchant"].lower()
    method = tx["method"].lower()
    hour = datetime.now().hour

    score = 0
    reasons = []

    if amount > 10000: score += 0.5; reasons.append("💰 Very high amount")
    elif amount > 5000: score += 0.3; reasons.append("💳 High amount")
    elif amount > 1000: score += 0.1; reasons.append("💵 Medium amount")

    if any(w in merchant for w in ["suspicious", "fraud", "scam", "fake"]):
        score += 0.4; reasons.append("🏪 Suspicious merchant")

    if method in ["wire_transfer", "crypto", "gift_card"]:
        score += 0.2; reasons.append("💳 Risky method")

    if hour < 6 or hour > 23:
        score += 0.1; reasons.append("🌙 Odd hour")

    score += random.uniform(0, 0.15)
    score = min(score, 1.0)
    is_fraud = score > 0.7

    return {
        "score": round(score, 3),
        "is_fraud": is_fraud,
        "reasons": reasons or ["✅ No significant risk factors"]
    }

In [ ]:
def explain_transaction(tx):
    prompt = (
        f"Analyze this transaction:\n"
        f"User: {tx['user_id']}\n"
        f"Amount: ${tx['amount']}\n"
        f"Merchant: {tx['merchant']}\n"
        f"Method: {tx['method']}\n"
        f"Is this risky? Explain.\n"
    )
    return explainer(prompt)[0]["generated_text"].split("Explain.")[-1].strip()

In [ ]:
def simulate(n=5):
    users = ["alice", "bob", "charlie", "diana", "eve"]
    merchants = ["amazon", "walmart", "starbucks", "suspicious_store", "fake_shop"]
    methods = ["credit_card", "debit_card", "wire_transfer", "crypto"]

    results = []
    for _ in range(n):
        tx = {
            "user_id": random.choice(users),
            "amount": round(random.uniform(10, 8000), 2),
            "merchant": random.choice(merchants),
            "method": random.choice(methods)
        }
        result = score_transaction(tx)
        tx.update(result)
        tx["explanation"] = explain_transaction(tx)
        results.append(tx)

    return pd.DataFrame(results)

In [ ]:
df = simulate(2)
display(df)

fraud_rate = df["is_fraud"].mean() * 100
print(f"🚨 Fraud Rate: {fraud_rate:.1f}%")